## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yinchuan,38.4681,106.2731,29.62,55,60,9.31,CN,1610570506
1,1,ushuaia,-54.8000,-68.3000,59.00,51,75,5.75,AR,1610570326
2,2,cape town,-33.9258,18.4232,75.00,64,0,6.91,ZA,1610570317
3,3,hofn,64.2539,-15.2082,39.97,90,100,18.34,IS,1610570487
4,4,fort saint james,54.4333,-124.2500,29.98,90,92,6.20,CA,1610570507


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_choice = float(input("Select your Minimum Temperateure Preference "))

Select your Minimum Temperateure Preference 70


In [4]:
max_choice = float(input("Select your Maximum Temperateure Preference "))

Select your Maximum Temperateure Preference 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
temp_selections = city_data_df.loc[(city_data_df["Max Temp"] >= min_choice) & (city_data_df["Max Temp"] <= max_choice)]
temp_selections.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,cape town,-33.9258,18.4232,75.00,64,0,6.91,ZA,1610570317
6,6,rikitea,-23.1203,-134.9692,77.92,77,47,14.34,PF,1610570392
8,8,atuona,-9.8000,-139.0333,77.52,78,53,15.93,PF,1610570508
15,15,cansahcab,21.1667,-89.1000,72.00,88,90,5.75,MX,1610570509
16,16,rio gallegos,-51.6226,-69.2181,71.60,24,0,6.91,AR,1610570481


In [6]:
# 4a. Determine if there are any empty rows.
temp_selections.count()

City_ID       258
City          258
Lat           258
Lng           258
Max Temp      258
Humidity      258
Cloudiness    258
Wind Speed    258
Country       254
Date          258
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
temp_selection_clean = temp_selections.dropna()
temp_selection_clean.count()

City_ID       254
City          254
Lat           254
Lng           254
Max Temp      254
Humidity      254
Cloudiness    254
Wind Speed    254
Country       254
Date          254
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = temp_selection_clean[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,cape town,ZA,75.00,-33.9258,18.4232,
6,rikitea,PF,77.92,-23.1203,-134.9692,
8,atuona,PF,77.52,-9.8000,-139.0333,
15,cansahcab,MX,72.00,21.1667,-89.1000,
16,rio gallegos,AR,71.60,-51.6226,-69.2181,
17,cayenne,GF,78.80,4.9333,-52.3333,
19,saint-philippe,RE,80.60,-21.3585,55.7679,
20,koumac,NC,77.67,-20.5667,164.2833,
22,kapaa,US,78.80,22.0752,-159.3190,
24,plettenberg bay,ZA,70.00,-34.0527,23.3716,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    long = row['Lng']
    
    
    params["location"] = f"{lat},{long}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=params).json()
    print (hotels)
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
        

{'html_attributions': [], 'next_page_token': 'ATtYBwJSKVU1mTibNV_Wh6_qH1ZyhtUhOsJ4VYTpcNhHG2ikjdPW1J7eHQd0eDP7SZcPleD8On_q8RykKOzcWMs5ClUPhA27nL2Epi6g8fS-8QV2XmClqHGTGfglRi494ktD3vO2IepUnT8QcQkwW3_2io3CuRuSp0PQQ0lUNRcKIbI-wGaVvsgr2VSwe2UiE6AENRqVd5JR0LJIG8LhCUCYz8P0isDJa7uVOi73Vl4dLNs36AonoEuuSLvmU-5kITYUyNYh-UgHeD4ElhhJgmMEOqfkRS59mvgPLwcIv1ymu1-gUOJeF11uMpXpETjdsoxxJXQKTR9Q3vFp6ucc1WjQkX6RQwIUkFxzNDB5VQd8aOTZ27ZxzCmOl9Ie1XdYaYcbTtjpUELhkCVx-XhMMZ8RveJTt5ecOAIyeb8aqgOFkjlBjNbuhktprObP', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -33.8996146, 'lng': 18.4120393}, 'viewport': {'northeast': {'lat': -33.8984771697085, 'lng': 18.4131622802915}, 'southwest': {'lat': -33.9011751302915, 'lng': 18.4104643197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Radisson Blu Hotel Waterfront, Cape Town', 'opening_hours': {'open_now': True}, 'photos': [{'height': 2560, 'html_attributions': ['<a href="https://maps

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -23.1206197, 'lng': -134.9687529}, 'viewport': {'northeast': {'lat': -23.1194598197085, 'lng': -134.9673886697085}, 'southwest': {'lat': -23.1221577802915, 'lng': -134.9700866302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': "Pension Maro'i", 'opening_hours': {'open_now': True}, 'photos': [{'height': 2304, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/111098332724302207930">luigia b.</a>'], 'photo_reference': 'ATtYBwLdTnlZKvjhK2KqellvtYbfmFRfdr3evgu-vRuEpm3UpPNNGKCd63UNffgOhjC7hEOmJ4-wpvJJ0J4Bdo7PUjRNFhpXyTu9BQyR4LbSupBJnDz0Ntrn6yt7kUvh3U5vfjth0sAZgc35oc1mW3stfNSjZb4rhWgcsl6cRs5bHnDnzNBI', 'width': 4096}], 'place_id': 'ChIJnW6Jh0aIJ54RV2HnC1r0Hjw', 'plus_code': {'global_code': '5487V2HJ+QF'}, 'rating': 4, 'reference': 'ChIJnW6Jh0aIJ54RV2HnC1r0Hjw', 'scope': 'GOOGLE', 'types': ['lodging', 'point_of_interes

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 21.1415493, 'lng': -89.0740156}, 'viewport': {'northeast': {'lat': 21.1431000302915, 'lng': -89.07250411970848}, 'southwest': {'lat': 21.1404020697085, 'lng': -89.07520208029149}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Hacienda San Mateo', 'opening_hours': {'open_now': False}, 'photos': [{'height': 3120, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/112308215533930496042">Laura Gomez</a>'], 'photo_reference': 'ATtYBwLwMz4OZnVVt8GSYrIDRhUmcPqEaB5IJ3C9hjHuJEZNddz4XL9yi5TBGIWIYx5flMC7d4eKFcw54bf1qSmLR9oN9BYvqT95h4lyYGP9ZiSdD-ImkAhCElCQPoa0tKZR0GaAA5pepdMTE4EVtWRzMsnBKPAL4Bo6moz5w1p0VDVpAssf', 'width': 4160}], 'place_id': 'ChIJaTSw4AofVI8RRb9GN37FZTs', 'plus_code': {'compound_code': '4WRG+J9 Cansahcab, Yucatan, Mexico', 'global_code': '76HG4WRG+J9'}, 'rating': 4.8, 'reference': 'ChIJaTSw4AofVI8RRb9GN37FZTs

{'html_attributions': [], 'next_page_token': 'ATtYBwKy3WPQutSD9tP91pO1oq_r6RZmA7OypmAJKTL0gEmlbZL3qgrYzLW43BA59hv3gtclV1mT9H6EQnrtCO8Yml_A2uBWxyEFP-z-JjM-8ByfQ4CqwfHOm50WfkL7AOFbbQ5dmMCA4mr5sYupHdAn4q-8pTWuWJlesPCsFIXMBaKGf5p526RBoK41vOmJEThuCNQsd0JoNoh3ZtNMkHslg7etHjSz8fZPInJH5V760red-WY-QXJvwIAsy2G_zI-fZlHlMlEHNQT0zuHlUCJ4Kn0ETe2XmUzTb_jrYzXwSuljAVI-GRIG3SH05rEaamjxoHqRqhzqzDRMJiSSqJrimYtzB_qjxLVmsHLUtrI2HpPIw6Q2QCOxQf0vFnSlhHdxdaLQOQ76kfq6_hLNGcY15k5kQiYQwdDOe-2fiRkLVhBF9VlwO0xwcNZf', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 4.9387955, 'lng': -52.3314083}, 'viewport': {'northeast': {'lat': 4.940208530291502, 'lng': -52.3300759697085}, 'southwest': {'lat': 4.937510569708498, 'lng': -52.33277393029149}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Hôtel Le Dronmi', 'opening_hours': {'open_now': True}, 'photos': [{'height': 3096, 'html_attributions': ['<a href="https://maps.google.com/maps/contri

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -21.358434, 'lng': 55.77032209999999}, 'viewport': {'northeast': {'lat': -21.3570810697085, 'lng': 55.7717046302915}, 'southwest': {'lat': -21.3597790302915, 'lng': 55.7690066697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Chambres d\'hôte "La Trinité"', 'photos': [{'height': 359, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117069031951999342120">Chambres d&#39;hôte &quot;La Trinité&quot;</a>'], 'photo_reference': 'ATtYBwIS6wTAoClatEmsDaNeb8jbDPpUKPMq9femcc3vOTiS5jrUClDdZwQSt9ZoZOFK69lBrOJel9hsJVbGdK4ssFQMtATNQqE_fA06r5_e7H0vEDNVjfFhVSlusYhOH6hSiFHBOFMA3VZodO75letFnQlj4hxQjbTSvSc93UgEKbfXtgaR', 'width': 550}], 'place_id': 'ChIJqSF_M5cUeCERuCMQ4H3g9oM', 'plus_code': {'compound_code': 'JQRC+J4 Saint-Philippe, Réunion', 'global_code': '5HCQJQRC+J4'}, 'rating': 3.3, 'reference': 'ChIJqSF_M5cUeCERuCMQ4H3g

...

{'html_attributions': [], 'next_page_token': 'ATtYBwJ7TLo5BJSAulGHgca9qmtlGqZukR4LjaYzsZXxjJj0SY8d-rt06Soq4iqJ29KUtP786mIBMzoRDSug1_tgzaWXPPKbW23Gc91elK0ZcpmuH-bZhwQWpcP3nAwMtdg4r5V7B9MqXoZV7P-5yXkkY8FRUurhlQwjTqEk0oyakF49WFNuviNlnwzRQV5B1yfzJchudi0sV_9fEwZwAGvqbtO2Hlhvkqb7FK0BTSvNJhqTU9_X-3FwtbMWoAqM2SEkBEX31ORAL5k924Uz-zGxxGHW9QiLi_uqPVq7chMZ9maLNFZSUkpVFbT6MbssCWnIQaSa5iJYZ2q7N3QF0WQwgFjizYRXCSglY3eFFDexCn0OgglHyCHLkgMnzbC3isIXEiBvXJyXnsTLoXEhGZeBrWsXq6-ALg_lEbQsXy5Cs4IpMNYlkfhFcVzK', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -23.66176430000001, 'lng': -70.4034135}, 'viewport': {'northeast': {'lat': -23.6602735197085, 'lng': -70.40205971970849}, 'southwest': {'lat': -23.6629714802915, 'lng': -70.4047576802915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Holiday Inn Express Antofagasta', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1367, 'html_attributions': ['<a href="https://ma

{'html_attributions': [], 'next_page_token': 'ATtYBwL_HuBNIURNcijb7FGd2uZAcVL9oaP159h7d3jk0aaLVjt6X7f5zC1MFfGYwtEgHTfUNqO05-2mZbIEgwR30dbyun12E9Zp0TSOVXbyDK0QMIOOsFCBDuaj1MLqXZLXmqDJKs_CUtNw1ujwcR58Tf5vXoMIi2imwBfh09ZAXfS232i9HmG4QdMGOXzoBpM6FGsQ-gtc7NeYpTtRemXgYnB7SpXYoFYB7P-G_GjSzOL_Yiaa6nyzXJKoWmgQKzdrkQ8oNCvxVxO5W9uXKVEa1liy1KhyvMJVyYBz7K3GqR3th5ztldMpfnB1A3c-47mESDvWlvEaosPKNO06zPR6Es53_tu0Bt5JPKy8tX55S4v1I-5_OL0_Ab9WSGuOFa1bL5NumDJITM9m0apGVgTFKumxWiWUv29rO7WcbjE8wleFx46fPKjIOZTg', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -14.8119956, 'lng': -39.0304268}, 'viewport': {'northeast': {'lat': -14.8105580197085, 'lng': -39.02909276970851}, 'southwest': {'lat': -14.8132559802915, 'lng': -39.0317907302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Pontal Praia Hotel', 'opening_hours': {'open_now': True}, 'photos': [{'height': 857, 'html_attributions': ['<a href="https://maps.google.com/maps/co

{'html_attributions': [], 'next_page_token': 'ATtYBwL5D1UdSdjXZf0P9JEKRhaxq39siM86beS6t6iuEAPS5Ekl_1SnA_dH80pt2vhOHpZSDi8cutSB309SaTJc_ONpluurXeFi69CdDlFeXtwzt5EtAKihlBImp2fXqJCP58Ks2tpOkEVd2FwVrh54ZhzdVgPgpmCUXy5MJNskQFAdLbjxZcNrXtPh8TN04iNNuOoOMSsE-5uRigjNMzjrs7tdYnjIu-Q4oI4VOdHiy2bcTkhNdc363DZeJxSsacNLbQULwrNfB6dmhu58wOWCr_GvJ_FwUil-yI2rfWni2fDVX-Pl5RRIzCkaD5DJFkQ0O4zPaxOmXSZRtHKp1wC2gOZ3-7CGP6Hp3Pv7fEbXcKBMd9cCRHXoToEISQtVpkyYD6o1TBsUM25TOxFQLAGIXItihNsQMTIJNaKK5km03qR0vMXAPRiG8uwA', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -35.61861029999999, 'lng': 174.538681}, 'viewport': {'northeast': {'lat': -35.61741136970849, 'lng': 174.5401614302915}, 'southwest': {'lat': -35.62010933029149, 'lng': 174.5374634697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Pacific Rendezvous Motel Resort', 'photos': [{'height': 2268, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/1122769483

{'html_attributions': [], 'next_page_token': 'ATtYBwLgf5KpkQL04zDTAlATQXh2RRyvsh4bxy0H4jd7OyBj3UzEVD3m3M1_TRIrMq2QKhGTAwNfRbyieX-DS2IjIzWaRitUOOoFE-2ilcZh8EUoLquNYMzmVmYu5mKKhLVbhrIwpc4Z_sHPkdBAAHX1G7GIMBDGgFg36YEqgr7io2kVn7BacGquGvb74-poPxldpaejmD9uw3NB6R9K4pyBUg8Dd0Kwr9GgmWKFQbOy9_PRVZQTmy6bwl1MuviSYFnWdkMT8t5lDLOHhswipa9cpV0nMa9NBvMhBPiExS0WrhVOYujd2l9drf7iJnL0b09rwIhFYkFfKIGLRqEVSVF-AsWYMgNu2ElQe0Ad8QGevoSW9JC6kj_cmb4500l9ndncRY83gAeW4xvB8t93JR7IzgPXA9G5X9sXw8YvuQOE7jJphfymA9qW0_AU', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 12.580776, 'lng': -81.68974299999999}, 'viewport': {'northeast': {'lat': 12.5821164802915, 'lng': -81.6883522197085}, 'southwest': {'lat': 12.5794185197085, 'lng': -81.69105018029151}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Hotel Arena Blanca', 'opening_hours': {'open_now': True}, 'photos': [{'height': 3406, 'html_attributions': ['<a href="https://maps.google.com/map

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 1.3294762, 'lng': 172.9776756}, 'viewport': {'northeast': {'lat': 1.330816180291502, 'lng': 172.9789545802915}, 'southwest': {'lat': 1.328118219708498, 'lng': 172.9762566197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Tarawa Boutique Hotel', 'photos': [{'height': 2268, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/114337342713429813600">Lucas Wall</a>'], 'photo_reference': 'ATtYBwIzBgKFQXw1PH1OFAqRNPQ8V8rZJZ0r0ulcsZ-Z0YXngy5joL53I0P9epUa01zJlI7PO1t0hxndDYM4liAJA1lKZfu_GPHYABbSDKaCs-MX2T6wNCkvZGue6bx4NyYcaVmOSezNXRgAa2Zm0D5wsdfjn7PwMa5Gch1zQ3ppPdKEX_W9', 'width': 4032}], 'place_id': 'ChIJgSUvg5F8ZGURsu1ICOnPHIA', 'plus_code': {'compound_code': '8XHH+Q3 Bairiki, Kiribati', 'global_code': '6VHJ8XHH+Q3'}, 'rating': 3.3, 'reference': 'ChIJgSUvg5F8ZGURsu1ICOnPHIA', 'scope': 'GOOGLE', 'types': ['lodging', 'po

In [10]:
# 7. Drop the rows where there is no Hotel Name.


In [11]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

NameError: name 'output_data_file' is not defined

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
